
#  Введение в мультиагентные системы с LLM

Разберемся как современные мультиагентные системы (MAS) используют большие языковые модели (LLM) для кооперации, самоорганизации и решения сложных задач.

---

## Цели ноутбука:

- Понять базовую архитектуру и принципы работы MAS.
- Увидеть, как LLM используются в роли агентов.
- Познакомиться с современными MAS-фреймворками.


In [57]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML, Image

import base64
import io, requests
from PIL import Image as im
import matplotlib.pyplot as plt

# Мультиагентные системы (MAS) и их эволюция с LLM

Первые обзоры были предоставлены в статье 
[Large Language Model based Multi-Agents: A Survey of Progress and Challenges](https://arxiv.org/pdf/2402.01680)


### 🔍 Именно в этой статье исследователи отмечают, что переход от одиночного LLM-агента к мультиагентным системам позволил достичь успехов в сложном коллективном решении задач и симуляции миров.

#### Идея состоит в том, что несколько специализированных LLM-агентов (далее мы будем их называть просто агентами), наделённых разными ролями и знаниями, могут координировать действия и совместно планировать решения, наподобие команды людей. По данным обзора, такие системы уже применяются в самых разных доменах – от разработки программного обеспечения и управления роботами до социальных симуляций и игр


<!-- <img src="./pics/start_of_mas.png" width="65%"> -->

![ALT](./pics/start_of_mas.png "nazvanie")



[Large Language Model based Multi-Agents: A Survey of Progress and Challenges](https://arxiv.org/pdf/2402.01680)


## 🔍 Определение MAS
**Мультиагентная система на основе LLM**– это подход, при котором несколько автономных агентов, управляемых большими языковыми моделями, работают совместно для достижения общей цели.


![ALT](./pics/arch_1st.png "nazvanie")


[Large Language Model based Multi-Agents: A Survey of Progress and Challenges](https://arxiv.org/pdf/2402.01680)

### Агентов начали использовать в иммитации сложных взаимодействий и социальных процессов посредством множества LLM. 

[Simulating Strategic Reasoning: Comparing the Ability of Single LLMs and
Multi-Agent Systems to Replicate Human Behavior](https://arxiv.org/pdf/2402.08189#:~:text=Our%20evaluation%20shows%20that%20the,of%20the%20time)

#### Авторы этой работы отмечают успехи в симуляции человеческого общества силами LLM-агентов и на этой основе строят новые симуляции. Важный вывод из исследования  – **мультиагентная симуляция на LLM может правдоподобнее воспроизводить человеческую логику, чем одиночная модель.**

#####  **Игра ультиматум.**
В игре участвуют два игрока: A и B. Игроку A выдается определённая сумма денег. Затем ему предлагается поделить эту сумму между собой и игроком B в любой пропорции, после чего игрок B может либо принять долю, предложенную игроком А, и тогда сделка совершается, либо отказаться. Во втором случае оба игрока лишаются выигрышей и остаются ни с чем. При этом вся информация об игре и её правила известны обоим участникам заранее.

##### **Цель исследования**: проверить, способны ли большие языковые модели (LLM) имитировать стратегическое поведение людей в социальных ситуациях. Авторы используют классический «ультиматум» – игру, где люди часто ведут себя иррационально (например, наказывают несправедливого партнёра себе в ущерб) . Это позволяет выяснить, сможет ли ИИ воспроизвести такие сложные стратегии с учётом социальных норм.


##### **Актуальность**: моделирование человеческого поведения крайне сложное, но важное при разработке политик и систем . Если LLM смогут реалистично симулировать, как разные люди реагируют на новые правила (особенно не сугубо рационально), это даст инструмент для предварительного «прожига» идей и прогнозирования последствий без реальных экспериментов 


![ALT](./pics/multi_vs_alone.png "nazvanie")


##### Вывод: 

- мультиагентный подход устранил проблемы, связанные с неполнотой рассуждений:

    - Каждая стратегия детально проработана для своей роли, и агенты строго ей следуют. 
    - Одиночный GPT часто допускал логические ошибки, что делало симуляцию ненадёжной.

<img src="./pics/flow.png" width="65%">


[review](https://newsletter.victordibia.com/p/multi-agent-llm-applications-a-review)

## Архитектура MAS
- 🧑‍💻 **Agentic workflows (сами агенты)**

    - Современные мультиагентные системы на базе больших языковых моделей строятся как автономные agentic процессы – циклы из планирования действий и оценки результатов. По сути, подход состоит в переводе человеческой логики решения задач в структурированные подсказки для модели, благодаря чему LLM сама планирует шаги, задействует инструменты и итеративно улучшает решения

- 💬 **Взаимодействие агентов** 

    - Агенты общаются друг с другом с помощью естественного языка, обмениваясь сообщениями в формате «вопрос-ответ». Например, один агент может выступать в роли пользователя или инициатора задачи, а другой – в роли исполнителя или помощника. Они поочерёдно генерируют сообщения и реагируют, постепенно вырабатывая решение. Подобное ролевое взаимодействие позволяет разделять задачу на диалоговые шаги

- 🧠 **Планирование и координация**

    - Для решения сложных задач вводится иерархия агентов. Часто используется связка «планировщик – исполнитель»: отдельный агент-планировщик (или модуль планирования) анализирует цель пользователя и разбивает её на упорядоченный список подзадач . Далее один или несколько агентов-исполнителей берут подзадачи по очереди, выполняя каждую и возвращая результат планировщику. Планировщик может пересматривать план по мере поступления результатов – это напоминает работу менеджера, который следит за прогрессом и при необходимости корректирует действия команды.

- ⚙️ **Использование инструментов**

    - Ключевая возможность LLM-агентов – это выход за пределы текстового диалога путём вызова внешних инструментов или API. Агент может осуществлять поиск информации, вызывать калькуляторы, базы знаний, запускать код и т.д.

- 🤖 **Память и контекст**

    - Чтобы эффективно работать в длительном сценарии, агенту нужна память о том, что уже было сделано или обсуждено. Память может быть организована как просто накопление текста диалога (в контексте) или как более сложное хранилище знаний. Многие реализации включают вектурную базу знаний: туда сохраняются эмбеддинги, полученных данных, принятых решений, и при необходимости агент может извлекать оттуда релевантную информацию по ходу работы. Таким образом обеспечивается состояниe  мультиагентной системы – агенты не забывают, к чему они пришли на предыдущих шагах, даже если это вышло за рамки их краткосрочного контекста. 

**Типичный рабочий процесс может выглядеть так:**

 - **Задача разбивается на подзадачи и распределяется между агентами по их специализации.**

 - **Каждый агент решает свою часть, используя LLM для понимания и генерации текста, а также подключая внешние инструменты (поиск в интернете, базы данных, вычисления и пр.).** 
 - Агенты обмениваются промежуточными результатами и комментариями, уточняя общее решение. 
 
 - Итоговый ответ формируется путём объединения вклада всех агентов. 

# 🧩 Развитие MAS-фреймворков в эпоху LLM

## Этапы развития

### 🔹 До LLM (до 2023) 
MAS-фреймворки использовались в классическом виде, например, в робототехнике, симуляции и тд. Внутри систем не было языковых моделей


---

### 🔹 Первая волна (Весна 2023)

#### BabyAGI
- Вводит очередь задач и память.
- Агент способен "думать" о следующем шаге.
- Ещё не MAS в полном смысле.

##### Как работает BabyAGI:

1. Пользователь формулирует цель
2. По цели генерируются задачи
3. Выбирается приоритетная задача и отправляется на исполнение
4. Результаты исполнения фиксируются в векторной базе
5. На основе полученных результатов формируются новые задачи
6. Повторяем с шага 3

<img src="./pics/babyagi.png" width="65%">

[github](https://github.com/yoheinakajima/babyagi)

---

### 🔹 Вторая волна (Осень 2023 – 2024)

Теперь MAS-фреймворки создаются осознанно: с множеством агентов, ролевой системой и взаимодействием.

---

## 🚀 Современные MAS-фреймворки с LLM

### 1. **AutoGen**
- Разработан Microsoft Research.
- Чёткая архитектура и агентное взаимодействие.

### 2. **CrewAI**
- Агенты с ролями, задачами и пайплайнами.
- Используется для генерации текста, отчетов, анализа.
- Прост в использовании.

### 3. **LangGraph**
- Архитектура на графе переходов.
- Поддерживает состояния, ветвления, контроль диалога.
- Подходит для сложных workflow.


---

## 📊 Таблица сравнения фреймворков

| Фреймворк   |   Год | Подход               | Сильные стороны                   | Применение           |
|:------------|------:|:---------------------|:----------------------------------|:---------------------|
| AutoGen     |  2023 | Чат-группа           | Надёжность, модульность           | Наука, дата-сценарии |
| CrewAI      |  2023 | Пайплайн с ролями    | Простота, задачи по ролям         | Контент, аналитика   |
| LangGraph   |  2024 | Граф состояний       | Асинхронность, контроль переходов | Сложные процессы     |

# Где используют агентов?

### Мультиагентные LLM-системы находят применение там, где требуется сложная координация или разные экспертные навыки:

- **Планирование и логистика**: агенты распределяют роли при организации путешествий, перевозок или закупок. Например, один агент ищет авиабилеты, второй – отели, третий – аренду транспорта. Совместно они «спланируют поездку» более эффективно, чем один универсальный агент. Аналогично в логистике: один агент может отслеживать склады, другой – маршруты доставки, третий – распределять заказы 

- **Разработка программного обеспечения**: несколько агентов могут совместно писать и анализировать код. Один агент генерирует шаблон программы, другой – проверяет безопасность, третий – пишет тесты. Исследования показывают успехи в автоматизации частей разработки ПО с помощью LLM-мультитула


- **Научные симуляции и моделирование**: мультиагентные LLM используются для моделирования сложных систем. Например, агенты имитируют поведение индивидов в социальной сети или участников экономической системы; каждый агент анализирует свой фрагмент, а совместно они обнаруживают глобальные закономерности. Также проводятся симуляции многоагентных систем в играх, политике или биологии. 

# Так как агенты "думают"?

# 🧠 Подходы к рассуждению в LLM: CoT и SPP

## 1. Chain-of-Thought (CoT) Prompting

**Chain-of-Thought (CoT)** — это стратегия, при которой LLM поощряется "думать вслух", то есть пошагово выводить логическое решение задачи перед финальным ответом.

<img src="./pics/COT.png" width="55%">

[Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/pdf/2201.11903)


### ➕ Преимущества:
- Разбивает задачу на логические шаги.
- Повышает точность на логических, арифметических и reasoning-задачах.

### ➖ Ограничения:
- Работает только как одиночный поток рассуждений.
- Нет самопроверки, критики, итерации.
- Требует ручных примеров (few‑shot)

### 📈 Эффективность:
CoT существенно улучшает производительность на задачах арифметики, символики, здравого смысла, особенно с большими LLM 


## 2. Solo Performance Prompting (SPP)

<img src="./pics/ssp.png" width="45%">

[Unleashing the Emergent Cognitive Synergy in Large Language Models: A Task-Solving Agent through Multi-Persona Self-Collaboration](https://arxiv.org/pdf/2307.05300)



###  🧠 SPP -  это метод, при котором один LLM симулирует работу нескольких агентов (персон), каждая из которых выполняет отдельную роль:

- 🧠 Writer: генерирует ответ.
- 🔍 Critic: проверяет.
- 🧑‍💼 Finalizer: улучшает и завершает ответ.

### ➕ Преимущества:
- Имитация когнитивной синергии внутри LLM.
- Самокритика и итеративные улучшения.
- Снижает количество фактических ошибок (галлюцинаций).

### ➖ Минусы:
- Лучше работает с GPT-4.
- Вычислительно дороже (больше токенов, больше вызовов).

## 📊 Сравнение CoT и SPP

| Характеристика           | Chain-of-Thought (CoT)         | Solo Performance Prompting (SPP)     |
|--------------------------|--------------------------------|--------------------------------------|
| Подход                   | Пошаговое мышление             | Мульти-агентность внутри одного LLM     |
| Количество "агентов"     | 1                              | Симулируется 2–4+                    |
| Обратная связь           | Нет                            | Да (критика)                |
| Итерации                 | Ограниченные                   | Есть (перепроверка)                  |
| Требуемый LLM            | GPT-3.5+                       | GPT-4 и выше                |
| Примерный prompt         | "Реши пошагово..."             | "Ты — команда экспертов..."          |
| Эффективность            | Средняя (но стабильная)        | Высокая, но затратная                |


#### Перспектива, в которой агенты смогут надежно помогать нам со  сложными задачами (например, создавать целые программные приложения, подавать налоговые декларации и т.д.), интересна и, возможно, близка. Сегодняшние LLM могли бы стать  строительными блоками таких агентов. При наличии доступа к инструментам (LLM + инструменты) и при работе в группах эти агенты могли бы решать сложные задачи, которые сегодня недоступны.
